<a href="https://colab.research.google.com/github/sammig6i/kaparthy-makemore-tuts/blob/main/makemore_part1/makemore_part1_E01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import torch
import torch.nn.functional as F

In [24]:
words = open("/content/names (1).txt", 'r').read().splitlines()

In [25]:
len(words)

32033

In [58]:
chars = ["."] + sorted(list(set("".join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
sstoi = {(s1, s2):i for i, (s1, s2) in enumerate([(s1, s2) for s1 in chars for s2 in chars])}
stoi["."] = 0
itos = {i:s for s, i in stoi.items()}
itoss = {i:(s1, s2) for (s1, s2), i in sstoi.items()}

In [59]:
xs, ys = [], []
for w in words:
  chs = ["."] + list(w) + ["."]
  for (ch1, ch2), ch3 in zip(zip(chs, chs[1:]), chs[2:]):
    ix1 = sstoi[(ch1, ch2)]
    ix2 = stoi[ch3]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('Number of examples: ', num)
print(f'xs size: {xs.shape}')
print(f'ys size: {ys.shape}')


Number of examples:  196113
xs size: torch.Size([196113])
ys size: torch.Size([196113])


In [60]:
for i in range(10):
  print(f"{itoss[xs[i].item()]} -> {itos[ys[i].item()]}")
  print(f"{xs[i]} -> {ys[i]}")

('.', 'e') -> m
5 -> 14
('e', 'm') -> m
148 -> 14
('m', 'm') -> a
364 -> 2
('m', 'a') -> .
352 -> 0
('.', 'o') -> l
15 -> 13
('o', 'l') -> i
417 -> 10
('l', 'i') -> v
333 -> 23
('i', 'v') -> i
265 -> 10
('v', 'i') -> a
603 -> 2
('i', 'a') -> .
244 -> 0


In [61]:
# init a 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((len(itoss),27), generator=g, requires_grad=True)

In [62]:
batches = torch.arange(0, 100, 1)
losses = []

xenc = F.one_hot(xs, num_classes=len(itoss)).float()
print(f"xenc size: {xenc.size()}")

for _ in batches:

  # forward pass
  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdim=True)
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  losses.append()




xenc size: torch.Size([196113, 729])
